# Experiments for the Energy Efficiency Master thesis in DYNAMOS

This Jupyter notebook is used for the experiments for the master thesis in DYNAMOS about energy efficiency.

FABRIC API docs: https://fabric-fablib.readthedocs.io/en/latest/index.html


## Step 1: Configure the Environment Configure the Environment (has to be done once in the Jupyter Hub environment), Create Slice, Setup K8s Cluster & Configure DYNAMOS

Before running this notebook, you will need to configure your environment using the [Configure Environment](../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook. Note: this has to be done only once in the Jupyter Hub environment (unless configuration is removed/deleted of course).

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

After following all steps of the Configuring Environment notebook, you should be able to run this notebook without additional steps.

Next, you will need to have setup the slice in FABRIC using the [Create Slice](../create_slice.ipynb) notebook.

Finally, configure the kubernetes environment using the [Configure Kubernetes](../k8s-cluster-setup/k8s_setup.ipynb) and [Configure DYNAMOS Specifically](../dynamos/DYNAMOS_setup.ipynb) notebooks.

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

## Step 2: Setup the Environment for this Notebook

### Step 2.1: Import FABRIC API and other libraries

In [ ]:
import json
import traceback
import datetime

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


### Step 2.2: Configure the parameters and variables
Can be used to set the corresponding slice and other variables used for subsequent cells.

In [ ]:
slice_name = 'DYNAMOS_EnergyEfficiency'
# Nodes:
node1_name = 'k8s-control-plane'
node2_name = 'dynamos-core'
node3_name = 'vu'
node4_name = 'uva'
node5_name = 'surf'

### Step 2.3: Extend the Slice before Running Experiments
This code extends the slice's end date to two weeks (the current maximum lease time) to make sure that the slice can be used for a longer period of time.

In [ ]:
# IMPORTANT: The function is working, but it does not save in the slice information afterwards, which is likely a bug in FABRIC. So, for now just skip this part, 
# the slice is created with a 2 weeks period, which should be enough and otherwise the steps to create it can be repeated
try:
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)
    # Print slice information for debugging purposes
    slice.show()
    # Print lease end before
    print(f"Lease End (UTC) Before        : {slice.get_lease_end()}")

    # See https://github.com/fabric-testbed/jupyter-examples/blob/main/fabric_examples/fablib_api/renew_slice/renew_slice.ipynb
    # Calculate the end date for now + x days
    end_date = (datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S %z")
    # Renew the slice with the calculated end date
    slice.renew(end_date)

    # Verify changes afterwards
    print(f"Lease End (UTC) After       : {slice.get_lease_end()}")
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Step 3: Execute Experiments
Make sure you followed the fabric/dynamos/DYNAMOS_setup.ipynb steps to upload the charts folder and configure DYNAMOS to allow redeploying and other steps for these experiments, such as loading the dynamos-configs.sh to run redeploy_structurally when an experiment goes wrong, etc.

### Step 3.1: Prepare Node for Experiments
This step prepares the node for the experiments, such as installing Python and creating a virtual environment. This step has to be performed once only to configure the node. After this step, the experiments can be executed (see next step).

In [ ]:
try:
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)
    # Get the correct node to run the experiment on: k8s-control-plane node, since we execute all the scripts there, and dynamos-core is required for the rest.
    control_node = slice.get_node(name='control')
    # Make sure the directory exists:
    control_node.execute("mkdir -p ~/experiments")
    # Upload required files for the experiments preparation (cannot do ~, this will cause No such file, need to do relative path from home of the node):
    control_node.upload_file(local_file_path="prepare_node.sh", remote_file_path="./experiments/prepare_node.sh")

    # Add necessary permissions and execute the script to prepare the node environment
    stdout, stderr = control_node.execute(f"chmod +x ./experiments/prepare_node.sh && ./experiments/prepare_node.sh")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

### Step 3.2: Execute/Run Actual Experiments
This step runs the actual experiments. To change the experiments, such as a different implementation target, modify the variables in the cell before executing the cell. For example, change the repetitions or the archetype.

Note: on local experiments, it was required to port-forward Prometheus to query Prometheus for the experiments data, however, here it is not necessary, since it is already exposed through NodePort and can be accessed on the node by a specific URL (added in constants.py). Furthermore, the SSH tunnels for Prometheus and Grafana for example (see fabric/dynamos/DYNAMOS_setup.ipynb) do not interfere with the experiments and they are not required to be closed during the experiments.

IMPORTANT: make sure the constants.py variables are correct, when recreating the Kubernetes environment or FABRIC slice, etc., the NodePort and other variables specific to FABRIC might change. See constants.py for an explanation on this specific for FABRIC, make sure the variables are set correctly for that, otherwise the experiment execution will not be able to execute data requests and prometheus exports for example.

Before executing the experiment, make sure to check in once in a while and extract the resulting files from the experiment executions. This explanation is given here so that the logs can run continuously below the python code and you can easily navigate back to this snippet to extract files. Furthermore, it is recommended to execute this from your local machine, since having too many notebooks running at the same time caused crashes before in the FABRIC Jupyter Notebook environment. Also, this avoids taking up too much space in the FABRIC Jupyter environment, which I also encountered before. Therefore, it is recommended to only run the experiments below in the notebook, and then follow the below commands on your local machine to extract the files from the node where the experiments run.
```sh
# Compress the folder on the remote host using SSH (see fabric/k8s/k8s_setup.ipynb for how to connect with SSH and from which local machine location), such as:
# (This uses tar, since that is already present on the ubuntu node). This uses the same config file used in fabric/dynamos/DYNAMOS_setup.ipynb.
ssh -i ~/.ssh/slice_key -F ssh_config_upload_script dynamos-node "cd ~/experiments && tar -czf experiments_data.tar.gz data/"
# Then in the same location, download the file to your local machine, such as:
scp -i ~/.ssh/slice_key -F ssh_config_upload_script dynamos-node:~/experiments/experiments_data.tar.gz .
# Then it should be present, and you move it to your desired location, such as in the correct experiments folder in energy-efficiency/experiments/data-fabric and upload it to GitHub
```
So, do this once in a while to save the results of the experiments to your local machine. Afterwards you can upload it to your GitHub for example, like explained above in the code snippet.
 
IMPORTANT: once in a while after making sure the results are saved to my local machine and added to GitHub, clean the node data folder by removing it entirely. For example, this can be done in between switching archetypes and/or implementations:
```sh
# SSH into the node.
# Go to the location that contains the data folder
cd ~/experiments
# Remove the data folder:
rm -r data
```

If an experiment fails, such as status 400 and DYNAMOS breaks (i.e. all subsequent requests return status 400 or something similar that fails), you can run the following command inside the k8s-control-plane node in an SSH session to redeploy structurally:
```sh
redeploy_structurally
```
Or after switching implementations (or even archetypes) you can also run that command to redeploy DYNAMOS with the new implementation (do not forget to upload the required new things on the node for new implementations, such as the charts folder, see fabric/dynamos/DYNAMOS_setup.ipynb).
Afterwards, you can restart the experiments with the below code, making sure to wait at least a few seconds after everything is up and running in the Kubernetes cluster to give it time to start back up.

In [ ]:
# Change these variables for different experiments, such as archetypes, implementations, etc. 
# See energy-efficiency/experiments/README.md for explanation on how to use the script and some examples. And see execute_experiments.py for options.
CURRENT_EXP_ARCHETYPE = "ComputeToData"
CURRENT_EXP_REPS = 10
CURRENT_EXP_NAME = "baseline"
try:
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)
    # Get the correct node to run the experiment on: k8s-control-plane node, since we execute all the scripts there, and dynamos-core is required for the rest.
    control_node = slice.get_node(name='control')
    
    # Upload required files for the experiments:
    control_node.upload_file(local_file_path="constants.py", remote_file_path="./experiments/constants.py")
    control_node.upload_file(local_file_path="execute_experiments.py", remote_file_path="./experiments/execute_experiments.py")
    
    # Run the experiment. This needs to run the python script to allow the output to be added in the notebook output, with a separate script that did not happen
    stdout, stderr = control_node.execute(
        (
            # Make the script executable
            f"chmod +x ./experiments/execute_experiments.py && "
            # Go to the corresponding location
            f"cd ~/experiments && "
            # Activate the venv
            f"source dynamos-env/bin/activate && "
            # Execute the actual experiments. Use -u to use unbuffered mode for stdout, stderr, and stdin, 
            # so that print() calls and logs from inside the python script appear live (or at least flush immediately after each action)
            f"python3 -u execute_experiments.py {CURRENT_EXP_ARCHETYPE} {CURRENT_EXP_REPS} {CURRENT_EXP_NAME}"
        )
    )

except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

# Run HFL experiments

In [ ]:
NUMBER_OF_NODES      = 4
CYCLES                              = 10
# CHANGE_POLICIES         = 10

try:
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)
    # Get the correct node to run the experiment on: k8s-control-plane node, since we execute all the scripts there, and dynamos-core is required for the rest.
    control_node = slice.get_node(name='control')
    
    # Upload required files for the experiments:
    control_node.upload_file(local_file_path="constants.py", remote_file_path="./experiments/constants.py")
    control_node.upload_file(local_file_path="execute_experiments_fl.py", remote_file_path="./experiments/execute_experiments_fl.py")
    
    # Run the experiment. This needs to run the python script to allow the output to be added in the notebook output, with a separate script that did not happen
    stdout, stderr = control_node.execute(
        (
            # Make the script executable
            f"chmod +x ./experiments/execute_experiments_fl.py && "
            # Go to the corresponding location
            f"cd ~/experiments && "
            # Activate the venv
            f"source dynamos-env/bin/activate && "
            # Execute the actual experiments. Use -u to use unbuffered mode for stdout, stderr, and stdin, 
            # so that print() calls and logs from inside the python script appear live (or at least flush immediately after each action)
            # f"python3 -u execute_experiments.py {LEARNING_RATE} {CYCLES} {CHANGE_POLICIES}"
            f"python3 -u execute_experiments_fl.py {NUMBER_OF_NODES} {CYCLES}"
        )
    )

except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

#### Additional information: Switching Implementations
To switch implementations between experiments, run the following steps:
1. Switch branches from GitHub to the implementation branch for fabric specifically.
2. Upload the charts folder from your local machine to the node (see fabric/dynamos/DYNAMOS_setup.ipynb manual steps for how to do this).
3. SSH into the node and load the utility functions in the SSH session (see fabric/dynamos/DYNAMOS_setup.ipynb manual steps for how to do this, in short: source ./dynamos-configs.sh)
4. Run the redeploy_structurally function in the SSH session on the node. 
Now you can continue the experiments with this new implementation. Do not forget to change the variables above for the experiment before running the experiments again!